# Quadratic Sieve Algorithm 

$\bf{\text{input}}$ $n,B$ 

$\bf{\text{output}}$ Prime factors of $n$ 


### Algorithm

${\bf 1}.$ $S\leftarrow \{-1,2,p_2,...,p_t\}$ such that legendre $(n,p_i)=1$ and $p_2<p_3<\dots<p_t\leq B$

${\bf 2}.$ $m \gets \lfloor \sqrt n \rfloor$

${\bf 3}.$ Collect $t+1$ pairs $(a_i,b_i)$, $x = 0, \pm1, \pm2 \cdots$. $i\gets 1$. While $i \le t+1,$ do:

${\quad \quad \bf 3.1}.$ $b=q(x)=(x+m)^2-n$. Check if b is $B$-smooth. If not, pick new $x$

${\quad \quad \bf 3.2}.$ If $b$ is $B$-smooth and $b \prod_{j=1}^t {p_j}^{e_{ij}}$, then $a_i\gets (x+m), b_i\gets b$, and $u_i=(u_{i1},u_{i2},\cdots,u_{it})$, where $u_{ij}=e_{ij} \mod 2$ for $1\le j \le t$

${\quad \quad \bf 3.3}.$ $i\gets i+1$

${\bf 4}.$ With linear algebra find a non-empty subset $T \subseteq \{1,2,\cdots.t+1\}$, such that $\sum_{i\in T} u_i = 0$

${\bf 5}.$ $x=\prod_{i\in T} a_i \mod n$

${\bf 6}.$ For each $j, 1\leq j \leq t, l_j = (\sum_{i\in T} e_{ij}/2)$

${\bf 7}.$ $y=\prod_{j=1}^t {p_j}^{l_j}\mod n$ 

${\bf 8}.$ If $x\equiv \pm y \mod n$, find another subset $T$ and go to step 5.

${\bf 9}.$ $d=\gcd(x-y,n)$ and return $d$


       

### Pseudo Code

In [29]:
# Initialization
# parameters

import primesieve
import math
%run auxiliary.ipynb

L=[]
i = 0
p,q,N=rsa_modulus(50)
print("bits of N",bits(N))

bits of N 48


In [30]:
B = Ln(N)**.55 + 500


print("N,p,q,B:",N,p,q,B)
#print("I,J,bits of J-I",I,J,bits(J-I))
#if bits(J-I)>25:
#    print("You may experience memory problems")

N,p,q,B: 142740025653739 14934119 9557981 851.028186548765


In [31]:
# step 1

P = primesieve.primes(B) 
P_QS = [-1,2]+[p  for p in P[1::] if legendre_symbol(N,p)==1]
print("P_QS:",P_QS)
t = len(P_QS)

P_QS: [-1, 2, 3, 5, 17, 23, 31, 41, 59, 71, 73, 79, 83, 89, 97, 101, 127, 131, 149, 173, 181, 191, 193, 199, 211, 223, 229, 239, 241, 251, 257, 263, 271, 281, 283, 307, 311, 313, 317, 337, 373, 379, 401, 419, 421, 431, 443, 449, 461, 467, 479, 487, 547, 563, 577, 587, 593, 599, 601, 641, 647, 661, 709, 733, 751, 787, 797, 823, 827, 829, 839]


In [32]:
# we use trial division
print("I will use trial division")
num_of_rows = t+15
num_of_cols = t
    
i = 1
x = 0                           #the x values are chosen in the order 0, ±1, ±2,..
change_x=1
a=[]                            #the list with B-smooth values x[i] of the form (x+m) for some x in [0,1,...,t+1]
    
m = math.floor(math.sqrt(N))
u = np.zeros( (num_of_rows, num_of_cols) )
e = np.zeros( (num_of_rows, num_of_cols) )
bs=[]    

while i<=num_of_rows:   #since i starts from 1 
    b = ((x+m)**2)- N   

    if is_smooth(b,P_QS):  #test using trial division by elements in factor_base whether b is pt-smooth
        a.append(x+m) 
        bs.append(b)
        b_set = [j for j in trial_division(b)]
            
        if b<0: 
            #if q(xi)<0 then the first factor in factor base is 1 representing -1
            u[i-1,0]=1 # i-1 because i starts from 1 
            e[i-1,0]=1
        for j in range(len(b_set)):
            #b_set[j][0] is the prime factor and with index we find its position, aka the column in matrix u                 
            u[i-1,P_QS.index(b_set[j])]= exponent(b_set,b_set[j])%2
            e[i-1,P_QS.index(b_set[j])]= exponent(b_set,b_set[j])           
        i = i+1   
    #pick new x
    if change_x==1:
        if x<0:
            x=x*(-1)
        x=x+1
    else:
        x=x*(-1)
    change_x=change_x * (-1)

I will use trial division


In [33]:
eche = row_echelon(u.transpose())
eche = eche.transpose() 

In [ ]:
C = initialization_for_C(eche)

for i in range(2**100):
    sol = pick_a_random_solution(C)
    
    if bits(N)<=80:
        F = factor_(bs,a,sol,N) 
    else:    
        BS = [smooth[x][0]**2 - N for x in range(len(smooth))]
        A = [smooth[x][0] for x in range(len(smooth))]
        F = factor_(BS,A,sol,N) 
    
    if F>1 and F<N:
        f2 = N / F
        print("A solution  was found")
        print("Factors: ",F,f2)
        break
        
if i == 2**10-1:
    print("no solution found")